In [ ]:
import pandas as pd

In [ ]:
#Constituent information
ci = pd.read_csv("data/cons.csv")
#Constituent email addresses 
ce = pd.read_csv("data/cons_email.csv")
#Constituent subscription status
css = pd.read_csv("data/cons_email_chapter_subscription.csv")

## Exercise 1
Produce a “people” file with the following schema. Save it as a CSV with a header line to the working directory.
![exercise1](images/exercise1.png)

In [ ]:
#Select necessary columns
cons_email = ce[['cons_email_id', 'cons_id', 'email', 'is_primary']]

cons_info = ci[['cons_id', 'subsource', 'create_dt', 'modified_dt']]

cons_sub_status = css[['cons_email_id', 'isunsub', 'chapter_id']]

In [ ]:
#Merge dataframes
cons_info_and_emails = pd.merge(cons_info, cons_email, how='left', on='cons_id')
with_sub_status = pd.merge(cons_info_and_emails, cons_sub_status, how = 'left', on = 'cons_email_id')

In [ ]:
#Filter chapter_id and is_primary
filter_df = with_sub_status[with_sub_status['chapter_id'] == 1]
filter_df = with_sub_status[with_sub_status['is_primary'] == 1]

In [ ]:
#Rename columns
people = filter_df[["email", "subsource", "isunsub", "create_dt", "modified_dt"]]
people = people.rename(columns={"subsource": "code", 
                       "isunsub": "is_unsub", 
                       "create_dt" : "created_dt", 
                       "modified_dt" : "updated_dt"})

In [ ]:
#Convert data types
people['email'] = people['email'].astype('str')
people['code'] = people['code'].astype('str')
people['is_unsub'] = people['is_unsub'].astype('bool')
people['created_dt'] =  pd.to_datetime(people['created_dt'], format='%a, %Y-%m-%d %H:%M:%S')
people['updated_dt'] =  pd.to_datetime(people['updated_dt'], format='%a, %Y-%m-%d %H:%M:%S')

## Exercise 2
Produce an “acquisition_facts” file with the following schema that aggregates stats about when people in the dataset were acquired. Save it to the working directory.
![exercise2](images/exercise2.png)

In [ ]:
#Select necessary column 
acquisition_facts = ci[['create_dt']]

In [ ]:
#Mutating create_dt column
acquisition_facts["create_dt"].replace('\\b(Sun|Mon|Tue|Wed|Thu|Fri|Sat)\\b,','',regex=True, inplace = True)
acquisition_facts["create_dt"].replace('[0-9]{2}:[0-9]{2}:[0-9]{2}', "", regex=True, inplace = True)

In [ ]:
#Rename create_dt to acquisition_date
acquisition_facts = acquisition_facts.rename(columns={"create_dt": "acquisition_date"})

In [ ]:
#Sort and count dates 
acquisition_facts = acquisition_facts.groupby(['acquisition_date'])['acquisition_date'].agg(['count']) 

In [ ]:
#Rename column
acquisition_facts = acquisition_facts.rename(columns={"count": "acquisitions"})